In [25]:
import os
from dotenv import load_dotenv
load_dotenv()  #load all the environment variables

True

In [27]:
os.environ['HUGGINGFACE_API_kEY']=os.getenv("HUGGINGFACE_API_kEY")#loading hugging face api key 

In [13]:
## Text Loader

from langchain_community.document_loaders import TextLoader

loader=TextLoader('speech.txt')
loader


In [20]:
#loading 
document=loader.load()
document


[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

In [23]:
#splitting into chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
documents=text_splitter.split_documents(document)
documents


[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_co

In [28]:
#importing library for embedding and vector database
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [29]:

#intializing embedding
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#intializing vectorDB where both embedding and chunked document are inserted
vectorstore=FAISS.from_documents(documents,embeddings)
vectorstore



/Users/sourabhmatali/Desktop/Agentic Aiworkspace/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#importing llm model
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="o1-mini")
print(llm)

In [ ]:
#creating chatprompttemplate
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

In [30]:
#here create_stuff_documents_chain  act as chain(|)
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

NameError: name 'llm' is not defined

In [ ]:
retriever=vectorstore.as_retriever()#path to go ahead vectorDB and ask anything

from langchain.chains import create_retrieval_chain #here create_retrieval_chain joins retriver and document_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

In [ ]:
result=retrieval_chain.invoke({"input":"Note that ChatModels receive message objects as input"})#here to test model

In [ ]:
result['answer']#finally answer